In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch


In [ ]:
def load_text_model(model_name="distilbert-base-uncased"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    return tokenizer, model

In [6]:
def predict_text(texts, tokenizer, model, device=None):
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=-1)[:, 1].cpu().numpy()
    return probs

In [7]:
# src/data_utils.py
import random
from datasets import Dataset

def make_simple_text_dataset():
    texts = [
        "This is a human written news short story about crops.",
        "Generated text The government gave 1000 to all citizens.",
        "Human message about community meeting tomorrow.",
        "AI generated news saying the village leader asked for money.",
        "Local market will be closed on Monday due to festival.",
        "Machine made article saying the school was closed by officials."
    ]
    labels = [0, 1, 0, 1, 0, 1]
    return Dataset.from_dict({"text": texts, "label": labels})


In [10]:
from transformers import TrainingArguments, Trainer

def tokenize_batch(batch, tokenizer):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

def train_text_baseline(output_dir="out"):
    tokenizer, model = load_text_model()
    ds = make_simple_text_dataset()
    ds = ds.map(lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=128), batched=True)
    ds = ds.rename_column("label", "labels")
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=2,
        num_train_epochs=2,
        logging_steps=5,
        save_strategy="no",
        report_to=[]
    )
    trainer = Trainer(model=model, args=args, train_dataset=ds)
    trainer.train()
     # Save the model and tokenizer after training
    trainer.save_model(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Training complete. Model and tokenizer saved to {output_dir}")

if __name__ == "__main__":
    train_text_baseline(output_dir="models/misinformation_detector") # Specify output directory


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 6/6 [00:00<00:00, 130.64 examples/s]


Step,Training Loss
5,0.627300


Training complete. Model and tokenizer saved to models/misinformation_detector


In [14]:
# ================================
# STEP 2: Evaluation & Inference
# ================================
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from sklearn.metrics import classification_report, confusion_matrix
import torch
import numpy as np

# Load trained model and tokenizer
model_path = "models/misinformation_detector"
model = DistilBertForSequenceClassification.from_pretrained(model_path)
tokenizer = DistilBertTokenizer.from_pretrained(model_path)

# Prepare test samples (replace with your actual test data)
texts = [
    "Government has launched a new education policy for rural areas.",
    "BREAKING: Minister caught stealing millions in new leaked video!!!",
    "Our CEO was seen begging in a viral video — click here to watch now!",
    "AI systems can generate false content that looks very real."
]
labels = [0, 1, 1, 0]  # 0 = real/human, 1 = AI/fake

# Tokenize and convert to tensors
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Run predictions
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1).numpy()

# Evaluation metrics
print("\n Classification Report:")
print(classification_report(labels, predictions, target_names=["Human", "AI/Fake"]))

# Confusion matrix
cm = confusion_matrix(labels, predictions)
print("\n Confusion Matrix:")
print(cm)

# Test individual inference
sample_text = "A viral video shows the president resigning immediately."
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, padding=True)
with torch.no_grad():
    logits = model(**inputs).logits
    prediction = torch.argmax(logits, dim=1).item()

label_name = "AI/Fake" if prediction == 1 else "Human"
print(f"\n Sample text: {sample_text}")
print(f"Predicted label: {label_name}")



 Classification Report:
              precision    recall  f1-score   support

       Human       0.67      1.00      0.80         2
     AI/Fake       1.00      0.50      0.67         2

    accuracy                           0.75         4
   macro avg       0.83      0.75      0.73         4
weighted avg       0.83      0.75      0.73         4


 Confusion Matrix:
[[2 0]
 [1 1]]

 Sample text: A viral video shows the president resigning immediately.
Predicted label: AI/Fake
